<!-- (evaluation)= -->
# WeatherBench 2 dataset retrieval

Whis notebooks contains basic examples of dataset retrieval. Datasets are stored on Google cloud, thus, briefly speaking, here is what one should do to get dataset:

1. Open it on Google cloud using link (I provided some, the rest can be found [in the "Ground truth datasets" section of the documentation](https://weatherbench2.readthedocs.io/en/latest/data-guide.html))
2. Specify timestamp of the dataset - after that, you will have a slice of the data at given point in time for every point in space
3. Specify variables and levels (in our basic case, we are only interested in the variables which are only measured near the surface, thus, no levels, only those variables)
4. Convert spefied dataset to Pandas DataFrame. Work with at as with pandas DataFrame.
5. Save in desired format, [compress it], and load to your machine.

In [1]:
# # Pip might complain about the Pandas version. The notebook should still work as expected.
# we don't need Weatherbench here, but can be useful in the future
# !pip install git+https://github.com/google-research/weatherbench2.git 

In [2]:
# import weatherbench2 # in this notebook we won't need it
import xarray as xr
import numpy as np
import pandas as pd

In [3]:
# # Run the code below to access cloud data on Colab. As we are working on our machines, we don't need it
# from google.colab import auth
# auth.authenticate_user()

# Creating tabular dataset
### Specify input datasets

Let's take a look at the datasets. Currently, the WeatherBench pipeline requires all input dataset to be stored as Zarr files.

In [4]:
# we need this dataset, the densest possible:
era_5_full_path = "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr"

Generally, we follow ECMWF's naming conventions for the input files.

* `time` [np.datetime64]: Time at forecast is initialized
* `lead_time` or `prediction_timedelta` [np.timedelta64]: Lead time
* `latitude` [float]: Latitudes from -90 to 90
* `longitude` [float]: Longitudes from 0 to 360
* `level` [hPa]: Pressure levels (optional)


Open the dataset:

In [5]:
era5_full = xr.open_zarr(era_5_full_path)

Specify timestamp (I picked one at random)

In [6]:
era5_full_single_day = era5_full.sel(time="2021-12-31T18:00:00.000000000")
era5_full_single_day # here is how time snapshot of the dataset is viewed - now it has 3 dimensions, longitude, latitude and levels.
# Each level containes some of the variables. But for now we will select variables which are measured only at the surface
# (only have longitude and latitude dimensions)

<xarray.Dataset>
Dimensions:                                           (latitude: 721,
                                                       longitude: 1440,
                                                       level: 13)
Coordinates:
  * latitude                                          (latitude) float32 90.0...
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float32 0.0...
    time                                              datetime64[ns] 2021-12-...
Data variables: (12/61)
    10m_u_component_of_wind                           (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    10m_wind_speed                                    (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    2m_dewpoint_temperature                           (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    2m_temperature                                    (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    ageostrophic_wind_speed                           (level, latitude, longitude) float32 dask.array<chunksize=(13, 721, 1440), meta=np.ndarray>
    ...                                                ...
    volumetric_soil_water_layer_1                     (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    vorticity                                         (level, latitude, longitude) float32 dask.array<chunksize=(13, 721, 1440), meta=np.ndarray>
    wind_speed                                        (level, latitude, longitude) float32 dask.array<chunksize=(13, 721, 1440), meta=np.ndarray>

Filter surface variables:

In [10]:
def filter_surface_variables(variables: list[str]) -> list[str]:
    """
    Returns only variables which are measured near the surface area -
    - they are recognised as those who have 2D shape of values
    (without the third dimension - `level`)
    """

    filtered_vars = filter(lambda var: len(era5_full_single_day[var].shape) == 2, variables)


    return list(filtered_vars)

all_variables = list(era5_full_single_day.variables)
surface_variables = filter_surface_variables(all_variables)
print(f"There are {len(surface_variables)} surface variables")

There are 47 surface variables


Convert xr.Dataset to familiar for evert data scientist pandas DataFrame. Be aware of the memory requered for this step! E.g., current grid has 721 $\times$ 1440 observations with 47 observations each $\implies $ ~ 4 bytes each variable $\implies$ (721 $\times$ 1440 $\times$ 47 $\times$ 4*8) > 1.5 Gb. This number can blow up easily.

In [11]:
def convert_time_snapshot_to_pandas_df(time_snapshot: xr.Dataset) -> pd.DataFrame:
    """
    Converts xr.Dataset to pd.Dataset, which has multiindex [Latitude, Longitude]
    and all corresponding variables for each coordinate
    """

    return time_snapshot.to_dataframe().drop("time", axis="columns")


df = convert_time_snapshot_to_pandas_df(era5_full_single_day[surface_variables])
df.head()

10m_u_component_of_wind  10m_v_component_of_wind  \
latitude longitude                                                     
90.0     0.00                      0.155021                 0.087519   
         0.25                      0.155021                 0.087519   
         0.50                      0.155021                 0.087519   
         0.75                      0.155021                 0.087519   
         1.00                      0.155021                 0.087519   

                    10m_wind_speed  2m_dewpoint_temperature  2m_temperature  \
latitude longitude                                                            
90.0     0.00              0.17802                      NaN      248.995667   
         0.25              0.17802                      NaN      248.995667   
         0.50              0.17802                      NaN      248.995667   
         0.75              0.17802                      NaN      248.995667   
         1.00              0.17802                      NaN      248.995667   

                    angle_of_sub_gridscale_orography  \
latitude longitude                                     
90.0     0.00                                0.78539   
         0.25                                0.78539   
         0.50                                0.78539   
         0.75                                0.78539   
         1.00                                0.78539   

                    anisotropy_of_sub_gridscale_orography  \
latitude longitude                                          
90.0     0.00                                         0.0   
         0.25                                         0.0   
         0.50                                         0.0   
         0.75                                         0.0   
         1.00                                         0.0   

                    boundary_layer_height  eddy_kinetic_energy  \
latitude longitude                                               
90.0     0.00                  279.227295                  0.0   
         0.25                  279.227295                  0.0   
         0.50                  279.227295                  0.0   
         0.75                  279.227295                  0.0   
         1.00                  279.227295                  0.0   

                    geopotential_at_surface  ...  total_column_water_vapour  \
latitude longitude                           ...                              
90.0     0.00                     -0.076172  ...                   2.421112   
         0.25                     -0.076172  ...                   2.421112   
         0.50                     -0.076172  ...                   2.421112   
         0.75                     -0.076172  ...                   2.421112   
         1.00                     -0.076172  ...                   2.421112   

                    total_precipitation_12hr  total_precipitation_24hr  \
latitude longitude                                                       
90.0     0.00                       0.000084                  0.000169   
         0.25                       0.000084                  0.000169   
         0.50                       0.000084                  0.000169   
         0.75                       0.000084                  0.000169   
         1.00                       0.000084                  0.000169   

                    total_precipitation_6hr  type_of_high_vegetation  \
latitude longitude                                                     
90.0     0.00                      0.000027                      0.0   
         0.25                      0.000027                      0.0   
         0.50                      0.000027                      0.0   
         0.75                      0.000027                      0.0   
         1.00                      0.000027                      0.0   

                    type_of_low_vegetation  volumetric_soil_water_layer_1  \
latitude longitude    

Here we can "flatten" our multiindexed dataframe so that every row now is the unique observation. Additionally, now every row has its coordinate value in longitude and latitude.

In [12]:
def transform_pd_from_multiindex_to_single_index(multiindex_df: pd.DataFrame) -> pd.DataFrame:

    return multiindex_df.reset_index()

In [13]:
df_plain = transform_pd_from_multiindex_to_single_index(df)
df_plain.head()

,latitude,longitude,10m_u_component_of_wind,10m_v_component_of_wind,10m_wind_speed,2m_dewpoint_temperature,2m_temperature,angle_of_sub_gridscale_orography,anisotropy_of_sub_gridscale_orography,boundary_layer_height,...,total_column_water_vapour,total_precipitation_12hr,total_precipitation_24hr,total_precipitation_6hr,type_of_high_vegetation,type_of_low_vegetation,volumetric_soil_water_layer_1,volumetric_soil_water_layer_2,volumetric_soil_water_layer_3,volumetric_soil_water_layer_4
0,90.0,0.00,0.155021,0.087519,0.17802,NaN,248.995667,0.78539,0.0,279.227295,...,2.421112,0.000084,0.000169,0.000027,0.0,0.0,-2.980232e-08,0.000003,-0.000006,2.980232e-08
1,90.0,0.25,0.155021,0.087519,0.17802,NaN,248.995667,0.78539,0.0,279.227295,...,2.421112,0.000084,0.000169,0.000027,0.0,0.0,-2.980232e-08,0.000003,-0.000006,2.980232e-08
2,90.0,0.50,0.155021,0.087519,0.17802,NaN,248.995667,0.78539,0.0,279.227295,...,2.421112,0.000084,0.000169,0.000027,0.0,0.0,-2.980232e-08,0.000003,-0.000006,2.980232e-08
3,90.0,0.75,0.155021,0.087519,0.17802,NaN,248.995667,0.78539,0.0,279.227295,...,2.421112,0.000084,0.000169,0.000027,0.0,0.0,-2.980232e-08,0.000003,-0.000006,2.980232e-08
4,90.0,1.00,0.155021,0.087519,0.17802,NaN,248.995667,0.78539,0.0,279.227295,...,2.421112,0.000084,0.000169,0.000027,0.0,0.0,-2.980232e-08,0.000003,-0.000006,2.980232e-08


Now we have plain tabular dataset, which can be easily transformed to any desired representation!

For now, let's compress it and save, then load and check that it is the same dataset!

In [14]:
df_plain.to_csv("era5_plain_compressed.csv.gz", index=False,
                compression={'method': 'bz2', 'compresslevel': 5}, )

In [15]:
!ls -lh ./

total 99M
-rw-r--r-- 1 fvelikon dpt_ext_searchportal_dep45091  713 Dec 24 15:04 conftest.py
drwxr-xr-x 4 fvelikon dpt_ext_searchportal_dep45091 4.0K Dec 24 15:04 docs
-rw-r--r-- 1 fvelikon dpt_ext_searchportal_dep45091  98M Dec 27 18:12 era5_plain_compressed.csv.gz
-rw-r--r-- 1 fvelikon dpt_ext_searchportal_dep45091  12K Dec 24 15:04 LICENSE
-rw-r--r-- 1 fvelikon dpt_ext_searchportal_dep45091   99 Dec 24 15:04 pyproject.toml
-rw-r--r-- 1 fvelikon dpt_ext_searchportal_dep45091 3.9K Dec 24 15:04 README.md
drwxr-xr-x 2 fvelikon dpt_ext_searchportal_dep45091 4.0K Dec 24 15:04 scripts
-rw-r--r-- 1 fvelikon dpt_ext_searchportal_dep45091 1.7K Dec 24 15:04 setup.py
-rw-r--r-- 1 fvelikon dpt_ext_searchportal_dep45091  30K Dec 27 18:00 test.ipynb
drwxr-xr-x 3 fvelikon dpt_ext_searchportal_dep45091 4.0K Dec 24 23:18 weatherbench2
-rw-r--r-- 1 fvelikon dpt_ext_searchportal_dep45091 344K Dec 27 18:12 weatherbench2_dataset_retrieval.ipynb
drwxr-xr-x 2 fvelikon dpt_ext_searchportal_dep45091 4.0K Dec 

In [16]:
df_plain_restored = pd.read_csv("era5_plain_compressed.csv.gz", compression={'method': 'bz2', 'compresslevel': 5})

In [18]:
def check_equality_safe(before_compression: pd.DataFrame, after_compression: pd.DataFrame):
    eps = np.finfo(np.float32).eps

    nonzero_rows, nonzero_cols = (np.abs(before_compression.values.astype(np.float32) - after_compression.values.astype(np.float32)) >= eps).nonzero()

    assert len(nonzero_rows) == 0 and len(nonzero_cols) == 0,  f"Cells with the differences: {list(zip(nonzero_rows, nonzero_cols))}"
    print("All values are the same before and after compression!")

In [26]:
check_equality_safe(df_plain, df_plain_restored)

All values are indistinguishable!


# Creating time-series dataset

Here is the code for expanding our graph tabular data to some period of time. Now it will have more compact structure with the shape: `[time, latitude, longitude, feature_dim]`. As such, we won't store latitude and longitude in the dataset, because this information is redundant. Instead, we will store additional arrays `latitudes` and `longitudes` which will have the same size as corresponding dimensions and from which we can easily restore the exact location. It is done because it crucially reduces the storage needed for the dataset (_e.g. in the previous example, directly storing latitude and longitude for each row costs ~64mb for single timestamp!_)

In [114]:
number_of_timestamps = 2

Specify starting date, time section and selected variables

In [115]:
time = pd.date_range("2000-01-01", freq="6h", periods=number_of_timestamps)
era_5_time_series = era5_full.sel(time=time)

In [116]:
era_5_time_series_surface_variables = era_5_time_series[surface_variables]
era_5_time_series_surface_variables

<xarray.Dataset>
Dimensions:                                           (time: 2, latitude: 721,
                                                       longitude: 1440)
Coordinates:
  * latitude                                          (latitude) float32 90.0...
  * longitude                                         (longitude) float32 0.0...
  * time                                              (time) datetime64[ns] 2...
Data variables: (12/47)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_wind_speed                                    (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_dewpoint_temperature                           (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    type_of_low_vegetation                            (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_1                     (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (time, latitude, longitude) float32 dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>

The most straightforward way to create dataset locally is to save in in `.npz` format with the keys as timestampes and values as dataframes for this particular timestamp:

In [117]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [145]:
from tqdm.auto import tqdm

def get_numpy_array_from_pd_df(pd_df: pd.DataFrame, latitude_dim_size: int, longitude_dim_size: int):
    
    """
    Transforms pandas Multiindex dataframe to numpy array with dimensions
                    `[latitude_dim_size, longitude_dim_size, feature_dim]`
    
    """
    
    # get proper order
    np_data: np.ndarray = pd_df.reorder_levels(order=["latitude", "longitude"]).values
    
    return np_data.reshape(latitude_dim_size, longitude_dim_size, -1)
    

def create_time_series_dataset(timeseries_xarray: xr.Dataset) -> None:
    """
    Create npz file for dataset with unique name `era5_<start time>_<finish_time>.npz`
    
    :timeseries_xarray: - xr.Dataset which contains the data
    
    returns npz file numpy wrapper
    
    """
    
    # get date and time of the first and the last timestamps"
    
    time_formatted = pd.to_datetime(timeseries_xarray["time"]).strftime('%Y_%m_%d_%H').values
    time_first, time_last = time_formatted[[0, -1]]
    
    dataset_name = f"era5_from_{time_first}_to_{time_last}"

    dataset_latitudes: np.ndarray = timeseries_xarray.latitude.values
    dataset_longitudes: np.ndarray = timeseries_xarray.longitude.values
    
    datasets: dict[str, np.ndarray] = {}
    
    for timestamp, time_string in tqdm(zip(timeseries_xarray["time"], time_formatted), total=len(time_formatted)):
        
        timestamp_df: pd.DataFrame = convert_time_snapshot_to_pandas_df(timeseries_xarray.sel(time=timestamp))
        
        timestamp_data_array: np.ndarray = get_numpy_array_from_pd_df(pd_df=timestamp_df, 
                                                          latitude_dim_size=len(dataset_latitudes), 
                                                          longitude_dim_size=len(dataset_longitudes),
                                                          )
        
        del timestamp_df
        
        datasets[time_string] = timestamp_data_array
        
        
    datasets.update(
        {"time": time_formatted, "latitude": dataset_latitudes, "longitude": dataset_longitudes}
        )
    
    np.savez_compressed(dataset_name, **datasets)
        
    
    # return opened .npz file to retrive keys later:
    
    return np.load(dataset_name + ".npz", allow_pickle=True)

In [146]:
era5_saved_time_series = create_time_series_dataset(era_5_time_series_surface_variables)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:15<00:00,  7.89s/it]


Test it:

In [159]:
for time_var in era_5_time_series_surface_variables["time"].values:
    
    time_formatted,  = pd.to_datetime([time_var]).strftime('%Y_%m_%d_%H')
    
    arr = era5_saved_time_series[time_formatted]
    loaded_df = pd.DataFrame(arr.reshape(-1, arr.shape[-1])) # flatten_array
    original_df = convert_time_snapshot_to_pandas_df(era_5_time_series_surface_variables.sel(time=time_var))
    
    check_equality_safe(loaded_df, original_df)

All values are indistinguishable!
All values are indistinguishable!
